# ResNet Recognition Model

by Flávia Carvalhido

### Train
- input (2): path to the image folder, path to the csv with image paths and classes
- output (1): trained model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import copy
import pandas as pd 
import shutil

cudnn.benchmark = True

Code adapted from PyTorch Tutorials : https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [10]:
class ResNetTrain:
    def __init__(self, img_folder_path="./train", img_csv="./train.csv"):
        self.img_folder_path = img_folder_path
        self.img_csv = img_csv

        # TODO: have way to test if folder has already needed structure and test if code can make the folder structure from csv
        # # Create directory with needed structure img_dir/class/img
        # os.mkdir('./temp_train')
        # df = pd.read_csv(self.img_csv)

        # self.class_names = df[df.columns[1]].unique()

        # for n in self.class_names:
        #     os.mkdir('./temp_train/'+n)

        # for _, row in df.iterrows():
        #     shutil.move(row[0], os.path.join('./temp_train/',row[1]))


        # Preprocess images (same for test)
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [
                                     0.229, 0.224, 0.225])
            ]),
        }

        image_datasets = {'train': datasets.ImageFolder(self.img_folder_path, data_transforms['train'])}

        self.dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=4, shuffle=True, num_workers=4)}
        self.dataset_sizes = {'train': len(image_datasets['train'])}
        self.class_names = image_datasets['train'].classes #override

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    def train(self): 

        model_ft = models.resnet18(pretrained=True)
        num_ftrs = model_ft.fc.in_features
        # size of each output sample is set to the number of classes
        model_ft.fc = nn.Linear(num_ftrs, len(self.class_names))

        model_ft = model_ft.to(self.device)

        criterion = nn.CrossEntropyLoss()

        # Observe that all parameters are being optimized
        optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

        # Decay LR by a factor of 0.1 every 7 epochs
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

        best_model_wts = copy.deepcopy(model_ft.state_dict())
        best_acc = 0.0

        for epoch in range(25):
            print(f'Epoch {epoch}/{25 - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase #FIXME: include validation??
            for phase in ['train']:
              
                model_ft.train()  # Set model to training mode
                

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in self.dataloaders[phase]:
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)

                    # zero the parameter gradients
                    optimizer_ft.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model_ft(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer_ft.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    exp_lr_scheduler.step()

                epoch_loss = running_loss / self.dataset_sizes[phase]
                epoch_acc = running_corrects.double() / self.dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model_ft.state_dict())

         
        # load best model weights
        model_ft.load_state_dict(best_model_wts)
        return model_ft


    def help(self):
        print("Class ResNetTrain will train a ResNet recognition model with the provided arguments (in order): the training images folder path and the training images csv path. After initializing the class with the previously mentioned arguments, just call the method \"train\" and a trained ResNet model will be returned.")

In [11]:
model = ResNetTrain("./archive/flowers", "")

model.train()

Epoch 0/24
----------
train Loss: 0.8377 Acc: 0.7086
Epoch 1/24
----------
train Loss: 0.5474 Acc: 0.8158
Epoch 2/24
----------
train Loss: 0.4136 Acc: 0.8654
Epoch 3/24
----------
train Loss: 0.3142 Acc: 0.8988
Epoch 4/24
----------
train Loss: 0.3138 Acc: 0.9039
Epoch 5/24
----------
train Loss: 0.2533 Acc: 0.9217
Epoch 6/24
----------
train Loss: 0.2130 Acc: 0.9365
Epoch 7/24
----------
train Loss: 0.1234 Acc: 0.9673
Epoch 8/24
----------
train Loss: 0.0895 Acc: 0.9792
Epoch 9/24
----------
train Loss: 0.0927 Acc: 0.9761
Epoch 10/24
----------
train Loss: 0.0773 Acc: 0.9798
Epoch 11/24
----------
train Loss: 0.0647 Acc: 0.9845
Epoch 12/24
----------
train Loss: 0.0662 Acc: 0.9849
Epoch 13/24
----------
train Loss: 0.0582 Acc: 0.9880
Epoch 14/24
----------
train Loss: 0.0646 Acc: 0.9859
Epoch 15/24
----------
train Loss: 0.0588 Acc: 0.9870
Epoch 16/24
----------
train Loss: 0.0533 Acc: 0.9882
Epoch 17/24
----------
train Loss: 0.0545 Acc: 0.9884
Epoch 18/24
----------
train Loss: 0.0

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Test 

- input (3),  trained model, path to the image folder, path to the csv with image paths and [optional: classes]
- output (2): 2D matrix of predictions (floats) for each image and class,  [optional, if ground truth is provided: accuracy]

In [ ]:
class ResNetTest:
    def __init__(self, trained_model, img_folder_path="./test", img_csv="./test.csv")
        self.img_folder_path = img_folder_path
        self.img_csv = img_csv

        